In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import scipy
import IPython
import string
import plotly
import plotly.figure_factory
import pickle

sys.path.append('../../')
sys.path.append('../')

import utils
import readers
import histogram
from histogram import describe_array, describe_arrays, describe_array_plotly
from readers.describe_bs_output_reader import DescribeBsOutputReader
from readers.patient_info import PatientInfo

In [ ]:
USE_PLOTLY = False

In [ ]:
output_dir = '/afs/csail.mit.edu/u/t/tzhan/NFS/script_output/describe_bs/'
outputReader = DescribeBsOutputReader(output_dir)
patientInfo = PatientInfo('../../../../patient_outcome_info/')

In [ ]:
sns.set(color_codes=True)
plotly.offline.init_notebook_mode(connected=True)

In [ ]:
# set read_output to False if pickled, saved results exist and you want to use them.
read_output = False

In [ ]:
if read_output:
    patient_to_bs_episode_lengths = {} # dict of lists of bs episode lengths for each patient
    patient_to_episode_to_burst_lengths = {} # dict of dicts of lists of burst lengths for each edf/episode for each patient
    new_sids = patientInfo.get_all_sids()

    for i, new_sid in enumerate(new_sids):
        print(i, new_sid)
        edfs_and_indices = patientInfo.get_edfs_and_indices(new_sid, max_num_hours=72)
        patient_episode_lengths = []
        episode_to_burst_lengths = {}
        for (edf, _, _) in edfs_and_indices:
            bs_episodes = outputReader.get_bs_episodes(edf)
            for episode_indices in bs_episodes:
                episode_start, episode_end = episode_indices
                patient_episode_lengths.append(utils.samples_to_min(episode_end-episode_start))
                bs_episode = bs_episodes[episode_indices]
                bs_episode_burst_lens = [len(burst['burst_data']) for burst in bs_episode]
                bs_episode_burst_lens = [utils.samples_to_sec(x) for x in bs_episode_burst_lens]
                episode_identifier = '{}_{}_{}'.format(edf, episode_start, episode_end)
                episode_to_burst_lengths[episode_identifier] = bs_episode_burst_lens
        patient_to_bs_episode_lengths[new_sid] = patient_episode_lengths
        patient_to_episode_to_burst_lengths[new_sid] = episode_to_burst_lengths
        IPython.display.clear_output()
    # save the things
    pickle.dump(patient_to_bs_episode_lengths, open('saved_output/patient_to_bs_episode_lengths.pkl', 'w'))
    pickle.dump(patient_to_episode_to_burst_lengths, open('saved_output/patient_to_episode_to_burst_lengths.pkl', 'w'))
else:
    patient_to_bs_episode_lengths = pickle.load(open('saved_output/patient_to_bs_episode_lengths.pkl'))
    patient_to_episode_to_burst_lengths = pickle.load(open('saved_output/patient_to_episode_to_burst_lengths.pkl'))

### How many patients with a bs episode longer than, eg, 60 mins?

In [ ]:
def get_bs_patients(min_episode_length):
    bs_patients = []
    for patient in patient_to_bs_episode_lengths:
        has_bs = False
        lens = patient_to_bs_episode_lengths[patient]
        for l in lens:
            if l > min_episode_length:
                has_bs = True
        if has_bs:
            bs_patients.append(patient)
    print '{} patients with 1+ episode longer than {} mins'.format(len(bs_patients), min_episode_length)
    return bs_patients
bs_patients = get_bs_patients(60)
bs_patients = get_bs_patients(45)
bs_patients = get_bs_patients(30)
bs_patients = get_bs_patients(-1)

# Num bs episodes per patient

In [ ]:
%matplotlib inline
num_episodes = []
for patient in patient_to_bs_episode_lengths:
    patient_episode_lengths = patient_to_bs_episode_lengths[patient]
    patient_num_episodes = len(patient_episode_lengths)
    num_episodes.append(patient_num_episodes)

if USE_PLOTLY:
    describe_array_plotly(num_episodes, 'Number of burst suppression episodes across patients', 'Number of burst suppression episodes', 'Number of patients', 
               rug_labels=patient_to_bs_episode_lengths.keys(), bin_size=5)
    describe_array_plotly(num_episodes, 'Number of burst suppression episodes across patients', 'Number of burst suppression episodes', 'Number of patients',
               rug_labels=patient_to_bs_episode_lengths.keys(), zoom=(0, 5))
else:
    num_episodes = np.array(num_episodes)
    describe_array(num_episodes, 'Number of burst suppression episodes across patients', 'Number of burst suppression episodes', 
                   'Number of patients', bins=55, save=True)

# Length of bs episodes

In [ ]:
xlabel = 'Burst suppression episode duration (mins)'
ylabel = 'Number of burst suppression episodes'

### All bs episode lengths

In [ ]:
all_bs_episode_lengths_flat = []
rug_labels = []
for patient in patient_to_bs_episode_lengths:
    all_bs_episode_lengths_flat += patient_to_bs_episode_lengths[patient]
    for i in range(len(patient_to_bs_episode_lengths[patient])):
        rug_labels.append(patient)

if USE_PLOTLY:
    describe_array_plotly(all_bs_episode_lengths_flat, 'Duration of burst suppression episodes across all patients', 
                   xlabel, ylabel, rug_labels=rug_labels, bin_size=1)
    describe_array_plotly(all_bs_episode_lengths_flat, 'Duration of burst suppression episodes across all patients', xlabel, ylabel, 
                   rug_labels=rug_labels, zoom=(0, 100))
else:
    all_bs_episode_lengths_flat = np.array(all_bs_episode_lengths_flat)
    describe_array(all_bs_episode_lengths_flat, 'Duration of burst suppression episodes across all patients', xlabel, ylabel, rug=False)
    describe_array(all_bs_episode_lengths_flat, 'Duration of burst suppression episodes across all patients', xlabel, ylabel, 
                   zoom=(0, 200), rug=False, save=True)

### Bs episode lengths per patient

In [ ]:
arrs = []
titles = []
for patient in patient_to_bs_episode_lengths:
    arrs.append(patient_to_bs_episode_lengths[patient])
    title = 'Length of bs episodes - {}'.format(patient)
    titles.append(title)
describe_arrays(arrs, titles, xlabel, ylabel, use_plotly=USE_PLOTLY)

# Duration of bursts

In [ ]:
xlabel = 'Burst duration (secs)'
ylabel = 'Number of bursts'

### All burst lengths

In [ ]:
burst_lengths_flat = []
for patient in patient_to_episode_to_burst_lengths:
    episode_to_burst_lengths = patient_to_episode_to_burst_lengths[patient]
    for episode in episode_to_burst_lengths:
        burst_lengths_flat += episode_to_burst_lengths[episode] 
if USE_PLOTLY:
    describe_array_plotly(burst_lengths_flat, 'Duration of bursts across all patients and episodes', xlabel, ylabel, rug=False)
    describe_array_plotly(burst_lengths_flat, 'Duration of bursts across all patients and episodes', xlabel, ylabel, 
                   zoom=(0, 5), bin_size=0.1, rug=False)
else:
    burst_lengths_flat = np.array(burst_lengths_flat)
    describe_array(burst_lengths_flat, 'Duration of bursts across all patients and episodes', xlabel, ylabel, rug=False)
    describe_array(burst_lengths_flat, 'Duration of bursts across all patients and episodes', xlabel, ylabel, 
                   zoom=(0, 5), rug=False, save=True)

### Burst lengths per patient

In [ ]:
arrs = []
titles = []
all_rug_labels = []
for patient in patient_to_episode_to_burst_lengths:
    burst_lengths_flat = []
    episode_to_burst_lengths = patient_to_episode_to_burst_lengths[patient]
    plot_rug_labels = []
    for episode in episode_to_burst_lengths:
        burst_lengths_flat += episode_to_burst_lengths[episode]
        for i in range(len(episode_to_burst_lengths[episode])):
            plot_rug_labels.append(str(episode))
    title = 'Length of bursts - {} - all episodes'.format(patient)
    arrs.append(burst_lengths_flat)
    titles.append(title)
    all_rug_labels.append(plot_rug_labels)
if USE_PLOTLY:
    describe_arrays(arrs, titles, xlabel, ylabel, rug_labels=all_rug_labels, use_plotly=USE_PLOTLY)
else:
    describe_arrays(arrs, titles, xlabel, ylabel, rug=False, use_plotly=USE_PLOTLY)

### Burst lengths per episode

In [ ]:
arrs = []
titles = []
for patient in patient_to_episode_to_burst_lengths:
    episode_to_burst_lengths = patient_to_episode_to_burst_lengths[patient]
    for episode in episode_to_burst_lengths:
        arrs.append(episode_to_burst_lengths[episode])
        title = 'Length of bursts - {} - Episode {}'.format(patient, episode)
        titles.append(title)
describe_arrays(arrs, titles, xlabel, ylabel, rug=False, use_plotly=False)